In [2]:
import warnings
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearnex import patch_sklearn, config_context
patch_sklearn()
warnings.filterwarnings('ignore')

/home/user/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
import os
ipynb_path = os.getcwd()
src_path = os.path.join(ipynb_path, 'src/')
input_path = os.path.join(ipynb_path,"input/")

In [4]:

import warnings
import os
import sys

import scipy.stats as spst

sys.path.append(src_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask
import dask.dataframe as dd
from windpowerlib.wind_speed import logarithmic_profile
from src.utils import uv_to_wsd # 윈도우에서는 앞에 src를 뺄것

In [5]:
power_2020 = pd.read_parquet(input_path + "dynamic_report_ewp02_2020_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2021 = pd.read_parquet(input_path + "dynamic_report_ewp02_2021_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2022 = pd.read_parquet(input_path + "dynamic_report_ewp02_2022_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power = pd.concat([power_2020, power_2021, power_2022], ignore_index=True)

gj_y = pd.read_parquet(input_path + "train_y.parquet").rename({'end_datetime': 'dt'}, axis=1)
ldaps = pd.read_parquet(input_path + "train_ldaps_gyeongju.parquet")

print("Power: ", power.shape)
print("train_y: ", gj_y.shape)
print("LDAPS: ", ldaps.shape)

Power:  (155528, 29)
train_y:  (52608, 4)
LDAPS:  (235818, 15)


In [7]:
! pip install seaborn
! pip install astral
! pip install swifter
! pip install scikit-learn-intelex
!pip install windpowerlib
! pip install lightgbm
!pip install scikit-learn-extra
! pip install xgboost

  Using cached swifter-1.4.0-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.8 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 27.9 MB/s eta 0:00:00m eta 0:00:010:00:01


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit

# yongmin's functions
from src.utils import DataConnector
from src.metric import NMAE
from src.data_processor import *

# model import
import xgboost as xgb
from xgboost import XGBRegressor


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [9]:
# 파이프라인 구성 및 적용
DataPipeline = Pipeline([
    ('uv_transform', UVTransformer('wind_u_10m', 'wind_v_10m')),
    ('wind_transform', WindTransformer('wind_speed', 10, 100, ldaps['surf_rough'].mean())),
    ('feature_engineering', FeatureTransformer()),
])

# 파이프라인을 이용하여 ldaps 데이터 변환
ldaps_transformed = DataPipeline.fit_transform(ldaps)

print(ldaps_transformed.shape)


(235818, 23)


In [10]:
average_ldaps = ldaps_transformed.drop('turbine_id', axis=1).groupby('dt').mean()
average_ldaps.columns = average_ldaps.columns.str.replace(r'[<>\[\]]', '_', regex=True)
average_ldaps.columns = average_ldaps.columns.str.replace(r'[^\w]', '_', regex=True)
average_ldaps.columns = average_ldaps.columns.str.replace(r'__+', '_', regex=True)

In [11]:
average_ldaps.reset_index(inplace=True)


In [12]:
average_ldaps['dt'] = pd.to_datetime(average_ldaps['dt']).dt.tz_localize(None)
gj_y['dt'] = pd.to_datetime(gj_y['dt']).dt.tz_localize(None)
avg_data = pd.merge(average_ldaps, gj_y, on='dt', how='inner')

In [13]:
avg_data_sorted = avg_data.sort_values(['dt', 'plant_name', 'energy_kwh'], ascending=[True, True, False])
avg_data_cleaned = avg_data_sorted.drop_duplicates(subset=['dt', 'plant_name'], keep='first')

In [14]:
avg_data_cleaned = avg_data.drop_duplicates(subset=['dt'], keep='first')
avg_data = avg_data_cleaned

In [15]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


def get_trasforms_datas(merged_data, numeric_columns, target):
    z_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()
    
    x_train = merged_data.loc[merged_data['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), numeric_columns]
    x_test = merged_data.loc[merged_data['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), numeric_columns]

    y_train = merged_data.loc[merged_data['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), target].shift(periods = -24)
    y_test = merged_data.loc[merged_data['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), target].shift(periods = -24)
    #y_train = y_train.dropna()
    #y_test = y_test.dropna()

    x_train = x_train.iloc[:-24]
    y_train = y_train.iloc[:-24]

    x_test = x_test.iloc[:-24]
    y_test = y_test.iloc[:-24]

    # Min-Max Scaling
    x_train_m = minmax_scaler.fit_transform(x_train)
    x_train_m = pd.DataFrame(x_train_m, columns=x_train.columns)
    x_test_m = minmax_scaler.transform(x_test)
    x_test_m = pd.DataFrame(x_test_m, columns=x_train.columns)

    # Standard Scaling
    x_train_z = z_scaler.fit_transform(x_train)
    x_train_z = pd.DataFrame(x_train_z, columns=x_train.columns)
    x_test_z = z_scaler.transform(x_test)
    x_test_z = pd.DataFrame(x_test_z, columns=x_train.columns)

    return x_train, x_test, x_train_m, x_test_m, x_train_z, x_test_z, y_train, y_test

In [16]:
from sklearn.cluster import KMeans

# 이제 특징 생성에 클러스터링 결과는 보지 않을 예정.
def addKmeansFeature(train_data, test_data):
    pd.options.mode.chained_assignment = None

    for n_clusters in range(2, 7):  # 2부터 6까지 클러스터 생성
        kmeans = KMeans(n_clusters=n_clusters, n_init=10)

        train_data[f'cluster_{n_clusters}'] = kmeans.fit_predict(train_data[['wind_speed', 'wind_direction']])
        
        test_data[f'cluster_{n_clusters}'] = kmeans.predict(test_data[['wind_speed', 'wind_direction']])

    return train_data, test_data
from sklearn.decomposition import PCA

def addPCAFeature(train_data, test_data):
    # PCA 적용할 특징 열 선택 (u, v 성분)
    wind_features = ['storm_u_5m', 'storm_v_5m', 'wind_u_10m', 'wind_v_10m', 
                     'wind_speed', 'wind_direction']
    
    # 훈련 데이터에서 PCA 학습
    pca = PCA(n_components=2)
    pca_train = pca.fit_transform(train_data[wind_features])
    
    # 훈련 데이터에 주성분 추가
    train_data['PC1'] = pca_train[:, 0]
    train_data['PC2'] = pca_train[:, 1]
    
    # 테스트 데이터에 PCA 적용
    pca_test = pca.transform(test_data[wind_features])
    test_data['PC1'] = pca_test[:, 0]
    test_data['PC2'] = pca_test[:, 1]

    # PCA 설명력 확인
    explained_variance = pca.explained_variance_ratio_
    print(f"PC1 설명력: {explained_variance[0]}")
    print(f"PC2 설명력: {explained_variance[1]}")

    return train_data, test_data

from sklearn_extra.cluster import KMedoids

def addKMedoidsFeature(train_data, test_data):
    pd.options.mode.chained_assignment = None

    for n_clusters in range(2, 7):  # 2부터 6까지 클러스터 생성
        kmedoids = KMedoids(n_clusters=n_clusters, random_state=42)

        # 훈련 데이터에 K-Medoids 클러스터링 적용
        train_data[f'medoid_cluster_{n_clusters}'] = kmedoids.fit_predict(train_data[['wind_speed', 'wind_direction']])

        # 테스트 데이터에 학습된 K-Medoids 모델 적용
        test_data[f'medoid_cluster_{n_clusters}'] = kmedoids.predict(test_data[['wind_speed', 'wind_direction']])

    return train_data, test_data


In [17]:
numeric_columns = avg_data.select_dtypes(include=['number']).columns.tolist()

In [18]:
x_train, x_test, x_train_m, x_test_m, x_train_z, x_test_z, y_train, y_test = get_trasforms_datas(avg_data, numeric_columns, 'energy_kwh')

In [19]:
x_train, x_test = addKmeansFeature(x_train, x_test)
x_train_m, x_test_m = addKmeansFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addKmeansFeature(x_train_z, x_test_z)
print('kmean 적용 완료')
x_train, x_test = addPCAFeature(x_train, x_test)
x_train_m, x_test_m = addPCAFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addPCAFeature(x_train_z, x_test_z)
print('pca 적용 완료')

x_train, x_test = addKMedoidsFeature(x_train, x_test)
x_train_m, x_test_m = addKMedoidsFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addKMedoidsFeature(x_train_z, x_test_z)
print('kmedoid 적용 완료')


kmean 적용 완료
PC1 설명력: 0.9982813596725464
PC2 설명력: 0.0008244868949986994
PC1 설명력: 0.7519216438993137
PC2 설명력: 0.11667728909910816
PC1 설명력: 0.333549415269656
PC2 설명력: 0.32870070013054326
pca 적용 완료
kmedoid 적용 완료


In [20]:
x_dict = {
    # 원본 데이터만 사용
    'original': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                 'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                 'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m'],

    # PCA 추가
    'pca_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                 'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                 'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'PC1', 'PC2'],

    # 클러스터(2~6) 추가
    'cluster_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                     'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                     'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                     'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6'],

    # PCA + 클러스터(2~6) 추가
    'pca_and_cluster': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                        'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                        'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                        'PC1', 'PC2', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6'],

    # 클러스터 개수에 따른 경우
    'cluster_2_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_2'],
    
    'cluster_3_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_3'],
    
    'cluster_4_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_4'],
    
    'cluster_5_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_5'],
    
    'cluster_6_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_6'],

    # KMedoids 추가
    'kmedoids_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                      'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                      'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                      'medoid_cluster_2', 'medoid_cluster_3', 'medoid_cluster_4', 'medoid_cluster_5', 'medoid_cluster_6'],

    # PCA + KMedoids 추가
    'pca_and_kmedoids': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                         'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                         'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                         'PC1', 'PC2', 'medoid_cluster_2', 'medoid_cluster_3', 'medoid_cluster_4', 'medoid_cluster_5', 'medoid_cluster_6'],

    # KMedoids 클러스터 개수에 따른 경우
    'medoid_cluster_2_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_2'],

    'medoid_cluster_3_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_3'],

    'medoid_cluster_4_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_4'],

    'medoid_cluster_5_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_5'],

    'medoid_cluster_6_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_6']

}


In [21]:
import torch

In [22]:
from src.deepTrain_roughVer.Analysis_WindTurbine.Model.RNNs import RNN, LSTM, GRU

In [23]:
from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim

In [34]:
from sklearn.metrics import r2_score, mean_absolute_error

def NMAE(y_true, y_pred):
    """Normalized Mean Absolute Error (NMAE) 계산 함수."""
    return mean_absolute_error(y_true, y_pred) / (sum(abs(y_true)) / len(y_true)) * 100

def train_rnn_model(save_dir, model, x_train, y_train, x_test, y_test, epochs=200, batch_size=30, lr=0.001):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
    x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        y_pred_list = []
        y_true_list = []
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, targets).item()
                y_pred_list.extend(outputs.cpu().numpy())
                y_true_list.extend(targets.cpu().numpy())

        y_pred_list = np.array(y_pred_list).flatten()
        y_true_list = np.array(y_true_list).flatten()

        mae = mean_absolute_error(y_true_list, y_pred_list)
        nmae = NMAE(y_true_list, y_pred_list)
        r2 = r2_score(y_true_list, y_pred_list)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), os.path.join(save_dir, f"{model.__class__.__name__}_best_model.pth"))

        print(f"Epoch [{epoch+1}/{epochs}]")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(test_loader):.4f}, MAE: {mae:.4f}, NMAE: {nmae:.4f}, R²: {r2:.4f}")

    torch.save(model.state_dict(), os.path.join(save_dir, f"{model.__class__.__name__}_final_model.pth"))


In [35]:
for key in x_dict.keys():
    # ====================
    # Min-Max 정규화 데이터
    # ====================
    x_train_m_selected = x_train_m[x_dict[key]]
    x_test_m_selected = x_test_m[x_dict[key]]

    # RNN 모델 학습
    rnn_model = RNN(input_dim=x_train_m_selected.shape[1], hidden_dim=128)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_avg/{key}_RNN_m/')
    train_rnn_model(save_dir, rnn_model, x_train_m_selected, y_train, x_test_m_selected, y_test)

    # LSTM 모델 학습
    lstm_model = LSTMModule(input_dim=x_train_m_selected.shape[1], hidden_dim=128)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_avg/{key}_LSTM_m/')
    train_rnn_model(save_dir, lstm_model, x_train_m_selected, y_train, x_test_m_selected, y_test)

    # GRU 모델 학습
    gru_model = GRUModule(input_dim=x_train_m_selected.shape[1], hidden_dim=128)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_avg/{key}_GRU_m/')
    train_rnn_model(save_dir, gru_model, x_train_m_selected, y_train, x_test_m_selected, y_test)

    # ====================
    # z-정규화 데이터
    # ====================
    x_train_z_selected = x_train_z[x_dict[key]]
    x_test_z_selected = x_test_z[x_dict[key]]

    # RNN 모델 학습
    rnn_model = RNNModule(input_dim=x_train_z_selected.shape[1], hidden_dim=64)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_avg/{key}_RNN_z/')
    train_rnn_model(save_dir, rnn_model, x_train_z_selected, y_train, x_test_z_selected, y_test)

    # LSTM 모델 학습
    lstm_model = LSTMModule(input_dim=x_train_z_selected.shape[1], hidden_dim=64)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_avg/{key}_LSTM_z/')
    train_rnn_model(save_dir, lstm_model, x_train_z_selected, y_train, x_test_z_selected, y_test)

    # GRU 모델 학습
    gru_model = GRUModule(input_dim=x_train_z_selected.shape[1], hidden_dim=64)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_avg/{key}_GRU_z/')
    train_rnn_model(save_dir, gru_model, x_train_z_selected, y_train, x_test_z_selected, y_test)


ValueError: Found input variables with inconsistent numbers of samples: [17370, 2223360]